In [1]:
import csv
import cv2
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation, Dropout, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [2]:
car_images = []
steering_angles = []
def process_image(image):
    return image

In [3]:
with open('data/data/driving_log.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            steering_center = float(row[3])

            # create adjusted steering measurements for the side camera images
            correction = 0.1     # this is a parameter to tune
            steering_left = steering_center + correction
            steering_right = steering_center - correction

            # read in images from center, left and right cameras
            path = 'data/data/' # fill in the path to your training IMG directory
            img_center = process_image(np.asarray(Image.open(path+row[0])))
            img_center_flipped = np.fliplr(img_center)
            img_left = process_image(np.asarray(Image.open(path+row[1])))
            img_left_flipped = np.fliplr(img_left)
            img_right = process_image(np.asarray(Image.open(path + row[2])))
            img_right_flipped = np.fliplr(img_right)

            # add images and angles to data set
            car_images.append(img_center)
            car_images.append(img_center_flipped)
            car_images.append(img_left)
            car_images.append(img_left_flipped)
            car_images.append(img_right)
            car_images.append(img_right_flipped)
            steering_angles.append(steering_center)
            steering_angles.append(-steering_center)
            steering_angles.append(steering_left)
            steering_angles.append(-steering_left)
            steering_angles.append(steering_right)
            steering_angles.append(-steering_right)
            
X_train = np.array(car_images)
y_train = np.array(steering_angles)
print(X_train.shape, y_train.shape)

(48216, 160, 320, 3) (48216,)


In [4]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5, activation="relu"))
model.add(MaxPooling2D())		#or can be MaxPooling2D((2, 2))
model.add(Convolution2D(6,5,5, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(Dense(75))
model.add(Dense(1))

In [5]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train,validation_split=0.2, shuffle=True, nb_epoch=7)

model.save('model_test.h5')

Train on 38572 samples, validate on 9644 samples
Epoch 1/7
38572/38572 [==============================] - 143s - loss: 0.0578 - val_loss: 0.0118
Epoch 2/7
38572/38572 [==============================] - 142s - loss: 0.0102 - val_loss: 0.0115
Epoch 3/7
38572/38572 [==============================] - 141s - loss: 0.0091 - val_loss: 0.0115
Epoch 4/7
38572/38572 [==============================] - 141s - loss: 0.0085 - val_loss: 0.0115
Epoch 5/7
38572/38572 [==============================] - 141s - loss: 0.0082 - val_loss: 0.0118
Epoch 6/7
38572/38572 [==============================] - 141s - loss: 0.0081 - val_loss: 0.0115
Epoch 7/7
38572/38572 [==============================] - 141s - loss: 0.0081 - val_loss: 0.0123
